In [2]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import plotly.express as px
import plotly.graph_objects as go
import datetime
import os

# Connect to Google Sheets
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('aquilacommercialsheets-923494a59a4b.json', scope)
client = gspread.authorize(credentials)

# Function to fetch fresh data
def get_fresh_data():
    # Open the spreadsheet
    spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1bzpRnUrpBH6l_zX7DtTypczZf5bpYVwqPUG3tzg2vec/edit?gid=1530779476#gid=1530779476'
    spreadsheet_id = spreadsheet_url.split('/')[5]
    sheet = client.open_by_key(spreadsheet_id).get_worksheet(0)
    df = pd.DataFrame(sheet.get_all_records())
    # Convert date columns to datetime with mixed format parsing
    df['DATE OF REQUIREMENT'] = pd.to_datetime(df['DATE OF REQUIREMENT'], format='mixed')
    df['DATE UPDATED'] = pd.to_datetime(df['DATE UPDATED'], format='mixed')

    # Use DATE UPDATED if it exists, otherwise use DATE OF REQUIREMENT
    df['EFFECTIVE_DATE'] = df['DATE OF REQUIREMENT']#df['DATE UPDATED'].fillna(df['DATE OF REQUIREMENT'])
    # Convert string values to numeric, removing commas and rounding
    df['REQUIRED SF (LOW)'] = pd.to_numeric(df['REQUIRED SF (LOW)'], errors='coerce')
    df['REQUIRED SF (HIGH)'] = pd.to_numeric(df['REQUIRED SF (HIGH)'], errors='coerce')
    df['REQUIRED SF (AVG)'] = (df['REQUIRED SF (LOW)'] + df['REQUIRED SF (HIGH)']) / 2
    
    return df

# Colors, similar to Dash version
COLORS = {
    'background': '#FFFFFF',
    'text': '#2C3E50', 
    'blue': '#00008B',
    'orange': '#DAA520',
    'light_gray': '#F8F9F9'
}

# Get data and process monthly aggregation
df = get_fresh_data()
df = df[df['EFFECTIVE_DATE'] >= pd.Timestamp('2024-12-31')]
monthly_data = df.groupby(pd.Grouper(key='EFFECTIVE_DATE', freq='ME')).agg({
    'REQUIRED SF (LOW)': ['sum', 'count'],
    'REQUIRED SF (HIGH)': 'sum',
    'REQUIRED SF (AVG)': ['mean', 'median']
}).reset_index()

# Flatten multi-level columns
monthly_data.columns = [
    'EFFECTIVE_DATE', 'REQUIRED SF (LOW)', 'REQUIRED SF (COUNT)', 
    'REQUIRED SF (HIGH)', 'REQUIRED SF (AVG)', 'REQUIRED SF (MEDIAN)'
]

# --- Figure 1: Sum of requirements (Line for LOW, HIGH) ---
fig1 = px.line(
    monthly_data,
    x='EFFECTIVE_DATE',
    y=['REQUIRED SF (LOW)', 'REQUIRED SF (HIGH)'],
    title='Monthly Total Square Footage Requirements',
    labels={'value': 'Square Footage', 'EFFECTIVE_DATE': 'Date'},
    color_discrete_sequence=[COLORS['orange'], COLORS['blue']]
)
fig1.update_layout(
    plot_bgcolor=COLORS['background'],
    paper_bgcolor=COLORS['background'],
    font=dict(family='Segoe UI', size=12),
    title={'font': dict(family='Segoe UI', size=24)},
    xaxis=dict(
        gridcolor=COLORS['light_gray'],
        tickformat='%b %Y',
        showgrid=True,
        dtick="M1"
    ),
    yaxis=dict(gridcolor=COLORS['light_gray']),
    margin=dict(t=100, b=50, l=50, r=50)
)

# --- Figure 2: Mean & Median of required SF (Line), and COUNT (Bar, secondary y-axis) ---
fig2 = go.Figure()

# Lines for Mean and Median
fig2.add_trace(go.Scatter(
    x=monthly_data['EFFECTIVE_DATE'],
    y=monthly_data['REQUIRED SF (AVG)'],
    mode='lines+markers',
    name='Avg SF (Mean)', 
    line=dict(color=COLORS['orange'])
))
fig2.add_trace(go.Scatter(
    x=monthly_data['EFFECTIVE_DATE'],
    y=monthly_data['REQUIRED SF (MEDIAN)'],
    mode='lines+markers',
    name='Avg SF (Median)',
    line=dict(color=COLORS['blue'])
))

# Bars for COUNT (secondary y-axis)
fig2.add_trace(go.Bar(
    x=monthly_data['EFFECTIVE_DATE'],
    y=monthly_data['REQUIRED SF (COUNT)'],
    name='Record Count',
    yaxis='y2',
    marker_color=COLORS['text'],
    opacity=0.3
))

fig2.update_layout(
    title={
        'text': 'Monthly Average Square Footage Metrics',
        'font': dict(family='Segoe UI', size=24)
    },
    plot_bgcolor=COLORS['background'],
    paper_bgcolor=COLORS['background'],
    font=dict(family='Segoe UI', size=12),
    xaxis=dict(
        title='Date',
        gridcolor=COLORS['light_gray'],
        tickformat='%b %Y',
        dtick="M1",
        showgrid=True
    ),
    yaxis=dict(
        title="Square Footage",
        gridcolor=COLORS['light_gray']
    ),
    yaxis2=dict(
        title="Count",
        overlaying='y',
        side='right',
        titlefont=dict(family='Segoe UI', size=12, color=COLORS['text']),
        tickfont=dict(family='Segoe UI', size=12, color=COLORS['text'])
    ),
    legend=dict(orientation="h", y=-0.2),
    margin=dict(t=100, b=50, l=50, r=50)
)

# --- Save to HTML files in "charts" directory ---
os.makedirs("charts", exist_ok=True)
fig1.write_html("charts/requirements_sf_total.html")
fig2.write_html("charts/requirements_sf_avg.html")

print("Saved charts/requirements_sf_total.html and charts/requirements_sf_avg.html")

Saved charts/requirements_sf_total.html and charts/requirements_sf_avg.html


In [3]:
df.head()

,DATE OF REQUIREMENT,DATE UPDATED,TENANT REP COMPANY,TENANT REP BROKER CONTACT,DEAL NAME,USE,MARKET,REQUIRED SF (LOW),REQUIRED SF (HIGH),TIMING,...,ASSIGNED BROKER,BUILDINGS SENT,SOURCE,INTERNAL NOTES,Timing,Timing Trim,CRAB TRAP/DITM,STATUS,EFFECTIVE_DATE,REQUIRED SF (AVG)
0,2025-12-02,2025-12-02,JLL,Bre Brown & Russell Young,,Office,NW,13000.0,16000.0,Q2 2027,...,"Max, Seth, Cody, Jon","The Campus at Arboretum, 9500 Arboretum, Brake...",Direct Email/Call,,,,Crab Trap,Active,2025-12-02,14500.0
1,2025-08-26,2025-12-02,S5 Advisory,Shelley Sakhuja,"Postman, Inc.",Office,"E, NW",4000.0,6000.0,,...,"Seth, Cody","Alto, Braker Pointe, Quarry Lake",,,,,DITM,Active,2025-08-26,5000.0
2,2025-03-05,2025-12-01,Cresa,Scott Studzinski,Apptronik,Office,"N, NW",60000.0,110000.0,,...,,,Broker,,,,DITM,Active,2025-03-05,85000.0
3,2025-12-01,2025-12-01,AQUILA,Jay Lamy,Undisclosed,Office,NE,65000.0,65000.0,,...,,,Broker,,,,DITM,Active,2025-12-01,65000.0
4,2025-12-01,2025-12-01,AQUILA,Chris Perry,Undisclosed,Office,NW,60000.0,90000.0,,...,,,Broker,,,,DITM,Active,2025-12-01,75000.0


In [30]:
import plotly.graph_objects as go

# Define Aquila golds and blues color palette (approximate HEX values)
AQUILA_COLORS = [
    "#002A5C",  # Deep Blue
    "#005BAB",  # Medium Blue
    "#69A3D2",  # Light Blue
    "#F2B134",  # Main Aquila Gold
    "#FFD166",  # Light Gold
    "#EDC87A",  # Warm Gold
    "#184D82",  # Secondary Blue
    "#33658A",  # Slate Blue
    "#F6D266",  # Pale Gold
    "#114477",  # Navy-Blue
    "#E7A932",  # Warm Deep Gold
    "#DA993A"   # Muted Gold
]

# Create a pie chart of REQUIRED SF (AVG) split by INDUSTRY
industry_data = (
    df.dropna(subset=['REQUIRED SF (AVG)'])
      .groupby('INDUSTRY')['REQUIRED SF (AVG)']
      .sum()
      .reset_index()
)
# Drop industries where REQUIRED SF (AVG) is 0
industry_data = industry_data[industry_data['REQUIRED SF (AVG)'] != 0]
# Drop the first row if it is a total row (i.e., INDUSTRY is blank or np.nan)
industry_data = industry_data[industry_data['INDUSTRY'].astype(str).str.strip() != '']

# Sort and select largest 7 industries, group rest into "Other"
industry_data_sorted = industry_data.sort_values(by='REQUIRED SF (AVG)', ascending=False)
top_n = 7
largest = industry_data_sorted.iloc[:top_n]
other = industry_data_sorted.iloc[top_n:]

if not other.empty:
    other_row = {
        'INDUSTRY': 'Other',
        'REQUIRED SF (AVG)': other['REQUIRED SF (AVG)'].sum()
    }
    pie_data = pd.concat([largest, pd.DataFrame([other_row])], ignore_index=True)
else:
    pie_data = largest

# Ensure the color list matches the number of categories (repeat if needed)
industry_colors = (AQUILA_COLORS * ((len(pie_data) // len(AQUILA_COLORS)) + 1))[:len(pie_data)]

fig3 = go.Figure(
    data=[
        go.Pie(
            labels=pie_data['INDUSTRY'],
            values=pie_data['REQUIRED SF (AVG)'],
            textinfo='label+percent',
            insidetextorientation='radial',
            hole=0.55,
            marker=dict(
                line=dict(color=COLORS['background'], width=2),
                colors=industry_colors
            )
        )
    ]
)
fig3.update_layout(
    title={
        'text': 'Tenant Demand by Industry',
        'font': dict(family='Futura LT Pro', size=24)
    },
    plot_bgcolor=COLORS['background'],
    paper_bgcolor=COLORS['background'],
    font=dict(family='Futura LT Pro', size=12),
    showlegend=False,
    width=820,
    height=650,
    margin=dict(t=100, b=80, l=50, r=50)
)
fig3.show()
fig3.write_html("charts/requirements_sf_avg_by_industry.html")
print("Saved charts/requirements_sf_avg_by_industry.html")


Saved charts/requirements_sf_avg_by_industry.html


In [32]:
# Bar chart: Total cumulative square feet requested by size range (horizontal bars)
# X axis: total SF, Y axis: size bins, with number of requirements labeled inside each bar

# Define new size range bins as specified
bins = [0, 14000, 40000, 100000, float('inf')]
labels = [
    '0-14k',
    '15k-39k',
    '40k-99k',
    '100k+'
]

# Bin the requirements
df['SIZE RANGE'] = pd.cut(df['REQUIRED SF (AVG)'], bins=bins, labels=labels, right=False)

# Group by size range: sum of requirements and count
size_group = df.groupby('SIZE RANGE').agg(
    **{
        'Total SF Requested': ('REQUIRED SF (AVG)', 'sum'),
        'Number of Requirements': ('REQUIRED SF (AVG)', 'count')
    }
).reset_index()

# Ensure all size bins are present in the right order
size_group['SIZE RANGE'] = pd.Categorical(size_group['SIZE RANGE'], categories=labels, ordered=True)
size_group = size_group.sort_values('SIZE RANGE').reset_index(drop=True)

# Plotly horizontal bar chart with annotation for number of requirements
# Add a light grey border on the bottom and left of the plot area by customizing axis lines
fig_size = go.Figure(
    data=[
        go.Bar(
            y=size_group['SIZE RANGE'],
            x=size_group['Total SF Requested'],
            orientation='h',
            marker_color=AQUILA_COLORS[:len(size_group)],
            text=size_group['Number of Requirements'],
            textposition='inside',  # <-- put inside the bars
            insidetextanchor='middle',
            hovertemplate=(
                'Size Range: %{y}<br>'
                'Total SF Requested: %{x:,.0f}<br>'
                'Number of Requirements: %{text}<extra></extra>'
            ),
        )
    ]
)
fig_size.update_layout(
    title={
        'text': 'Total Cumulative SF Requested by Size Range',
        'font': dict(family='Segoe UI', size=22)
    },
    xaxis=dict(
        title='Total Cumulative Requested SF',
        showgrid=True,
        zeroline=True,
        showline=True,  # enable border line
        linecolor='lightgrey',
        linewidth=2,
        mirror=False,
        ticks='outside'
    ),
    yaxis=dict(
        title='Requirement Size Range (SF)',
        tickmode='array',
        tickvals=labels,
        ticktext=labels,
        showline=True,  # enable border line
        linecolor='lightgrey',
        linewidth=2,
        mirror=False,
        ticks='outside'
    ),
    font=dict(family='Futura LT Pro', size=12),
    plot_bgcolor=COLORS['background'],
    paper_bgcolor=COLORS['background'],
    width=820,
    height=500,
    margin=dict(t=80, b=80, l=120, r=50)
)

# Tweak text font for annotations, and append "reqs" label
fig_size.update_traces(
    texttemplate='%{text} reqs',
    textfont=dict(family='Futura LT Pro', size=14)
)

fig_size.show()
fig_size.write_html("charts/requirements_by_size_range.html")
print("Saved charts/requirements_by_size_range.html")


C:\Users\NLin\AppData\Local\Temp\ipykernel_21408\713925717.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Saved charts/requirements_by_size_range.html


In [24]:
import aquila_graphing_tools
aquila_graphing_tools.commit_and_push_all("Tenant Demand by Industry graph")